In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **# Scenario 3: Dynamic Pricing Adjustment Based on Peak and Off-Peak Hours **

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define a function for demand response based on price
def demand_response(price):
    """Estimate demand based on price."""
    return max(100 - (price * 50), 0)  # Example: linear demand function

# Dynamic Pricing Simulation with Game Theory Approach
def dynamic_pricing_simulation(station_data):
    """
    Adjust prices based on peak and off-peak hours and simulate game theory interactions.
    
    Parameters:
    station_data (DataFrame): The DataFrame containing charging station data.
    
    Returns:
    DataFrame: Updated station data with new prices and demand.
    """
    peak_hours = [8, 9, 17, 18]  # Define peak hours
    off_peak_price = 0.25
    peak_price = 0.75  # Higher peak price

    # Adjust prices based on hours and simulate game theory
    for index, row in station_data.iterrows():
        hour = row['startTime']  # Assuming startTime is in 24-hour format
        if hour in peak_hours:
            station_data.at[index, 'dollars'] = peak_price
        else:
            station_data.at[index, 'dollars'] = off_peak_price
        
        # Game theory interaction: Adjust prices based on competitor pricing
        competitor_price = station_data['dollars'].mean()  # Average competitor price
        if station_data.at[index, 'dollars'] > competitor_price:
            # Reduce price slightly to attract more customers
            station_data.at[index, 'dollars'] -= 0.05
    
    # Calculate new demand based on the updated prices
    station_data['new_demand'] = station_data['dollars'].apply(demand_response)
    return station_data

# Sample station data for testing
# Create a sample DataFrame with random data including startTime
data = {
    'sessionId': range(1, 101),
    'kwhTotal': np.random.uniform(0, 10, 100),  # Random kWh totals
    'startTime': np.random.randint(0, 24, 100),  # Random start time in hours
    'dollars': np.random.uniform(0.5, 1.5, 100)  # Random initial prices
}
station_data = pd.DataFrame(data)

# Run the dynamic pricing simulation
updated_station_data = dynamic_pricing_simulation(station_data)

# Display updated data for verification
print(updated_station_data.head())

# Function to plot the prices and demands
def plot_price_vs_demand(station_data):
    """Plot the prices and demands of charging stations."""
    plt.figure(figsize=(12, 6))
    plt.plot(station_data['sessionId'], station_data['dollars'], label='Price ($)', color='blue', marker='o')
    plt.plot(station_data['sessionId'], station_data['new_demand'], label='New Demand', color='orange', marker='x')
    plt.title('Price and Demand of Charging Stations with Dynamic Pricing')
    plt.xlabel('Session ID')
    plt.ylabel('Value')
    plt.legend()
    plt.grid()
    plt.show()

# Call the plotting function to visualize the results
plot_price_vs_demand(updated_station_data)
